In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import time
import logging

import numpy as np
import matplotlib.pyplot as plt
import nidaqmx

In [3]:
import qcsapphire
import qt3rfsynthcontrol
import qt3utils.experiments.cwodmr
import qt3utils.nidaq

In [4]:
device_name = 'Dev1'
read_write_timeout = 10
clock_terminal = None
edge_input_terminal = 'PFI12'
edge_input_counter = 'ctr2'
clock_rate = 10000 #Hz
N_data_samples_to_acquire = 1000
daq_time = N_data_samples_to_acquire / clock_rate

In [31]:
nidaq_config = qt3utils.nidaq.EdgeCounter(device_name)
nidaq_config.reset_daq()

In [32]:
if clock_terminal is None:
    nidaq_config.configure_di_clock(clock_rate = clock_rate)

In [33]:
nidaq_config.configure_counter_period_measure(
    source_terminal = edge_input_terminal,
    N_samples_to_acquire_or_buffer_size = N_data_samples_to_acquire,
    clock_terminal = nidaq_config.clock_task_config['clock_terminal'],
    trigger_terminal = None,
    sampling_mode = nidaqmx.constants.AcquisitionType.FINITE)

nidaq_config.create_counter_reader()

In [34]:
nidaq_config.clock_task.start()


In [35]:
def read_daq_buffer(detector_reader,  N_samples,  read_write_timeout=10):
    nidaq_config.counter_task.start()
    data_buffer = np.zeros(N_samples)

    samples_read = detector_reader.read_many_sample_double(
                            data_buffer,
                            number_of_samples_per_channel=N_samples,
                            timeout=read_write_timeout)

    nidaq_config.counter_task.stop()
    return data_buffer, samples_read

In [36]:
def emitter(N):
    """return counts per second """
    for i in range(N):
        #data_sample = run_once(edge_detector_task, edge_detector_reader)
        data_sample, samples_read = read_daq_buffer(nidaq_config.counter_reader, N_data_samples_to_acquire)
        print(samples_read)
        yield data_sample.sum()/(samples_read / clock_rate)

In [39]:
d = emitter(10)

In [40]:
list(d)

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


[939190.0,
 939100.0,
 943480.0,
 944350.0,
 940230.0,
 940680.0,
 942580.0,
 938930.0,
 939770.0,
 944280.0]

In [41]:
nidaq_config.clock_task.stop()
nidaq_config.counter_task.stop()

In [42]:
nidaq_config.clock_task.close()
nidaq_config.counter_task.close()